# Interaction between characters

In [1]:
import sys
sys.path.insert(1, '../src')

from book_entities import *
from book_entities_embeddings_getters import *

## Book entities

In [2]:
# cleaned entities
textually_close_merged_book_cleaned_entities, lax_merged_book_cleaned_entities = get_book_entities(
    f'../data/book_entities/grouped_token_cleaned.csv')

In [3]:
for i in textually_close_merged_book_cleaned_entities:
    print(BookEntity.from_list_entity(i).get_shortname())

Comte Altamira
Amanda
M Appert
Bonaparte
Abbé Castanède
Caylus
Abbé Chélan
Croisenois
Danton
Mme Derville
M Descoulis
Dieu
Falcoz
M Fouqué
Vicaire Frilair
Geronimo
Abbé Julien Sorel
Mole Mathilde
Prince Korasoff
Estime
Père Marquis
Vicaire Maslon
Général Napoléon
Comte Norbert
M Pirard
Pitt
M Rênal
Mme Rênal
Mère Stanislas
Père Stanislas
M Tanbeau
M Valenod
Élisa


## Paragraphs

In [6]:
from bs4 import BeautifulSoup

In [7]:
# Reading the data inside the xml
# file to a variable under the name
# data
with open('../data/book/stendhal_rougenoir_xml.xml', 'r') as f:
	data = f.read()

In [8]:
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")

# Finding all instances of tag
# `unique`
b_p = Bs_data.find_all('p')
b_rend = Bs_data.find_all(rend=['right', 'right i'])

paragraphs = [i for i in b_p if i not in b_rend]
paragraphs = [p.contents[0] for p in paragraphs]

### dialogue et narration

In [128]:
narration = []
temp_dialogue = []
for p in paragraphs :
    if isinstance(p, str) :
        if p[0] == '–' :
            temp_dialogue.append(p)
        else :
            if len(temp_dialogue)>0 :
                narration.append(' '.join(temp_dialogue))
                temp_dialogue=[]
            narration.append(p)
        

pour chaque paragraphe chercher si il contient une entité, puis regarder toutes les entités qu'il contient 



### narration

In [129]:
def process_paragraph(p) :
    
    pattern1 = r"""[,.;@#?!&$\s:–]+"""
    pattern2 = r"""[-'’_]"""

    p = re.sub(r'|'.join((pattern1, pattern2)),
               " ",          # and replace it with a single space
               p.replace('\xa0', ' '), flags=re.VERBOSE)
    return [i.lower() for i in p.split(' ')[:-1] ]#if i.lower() not in french_stopwords]

In [130]:
def entity_in_paragraph(entity, paragraph):
    return [1 for address in entity.all_addresses if all(x.lower() in p for x in address.lower().split(' '))]

In [131]:
dict_interaction = {}
for paragraph in narration :
    p = process_paragraph(paragraph)
    paragraph_entities = [
        i for i in textually_close_merged_book_cleaned_entities if entity_in_paragraph(i, p)]
    
    if len(paragraph_entities)>=2 :
        key = frozenset(paragraph_entities)
        value = dict_interaction.get(key,[])
        value.append(p)
        dict_interaction[key] = value

In [132]:
len(dict_interaction)

140

In [58]:
dict_interaction

{frozenset({<book_entities.BookListEntity at 0x7f6ef4e0a970>,
            <book_entities.BookListEntity at 0x7f6ef4e0ab80>}): ['Ne vous attendez point à trouver en France ces jardins pittoresques qui entourent les villes manufacturières de l’Allemagne, Leipsick, Francfort, Nuremberg, etc. En Franche-Comté, plus on bâtit de murs, plus on hérisse sa propriété de pierres rangées les unes au-dessus des autres, plus on acquiert de droits aux respects de ses voisins. Les jardins de M.\xa0de\xa0Rênal, remplis de murs, sont encore admirés parce qu’il a acheté, au poids de l’or, certains petits morceaux de terrain qu’ils occupent. Par exemple, cette scie à bois, dont la position singulière sur la rive du Doubs vous a frappé en entrant à Verrières, et où vous avez remarqué le nom de Sorel, écrit en caractères gigantesques sur une planche qui domine le toit, elle occupait, il y a six ans, l’espace sur lequel on élève en ce moment le mur de la quatrième terrasse des jardins de M.\xa0de\xa0Rênal.',

## DataFrame of Interactions

In [133]:
entities = []
interactions = []
for k, v in dict_interaction.items() :
    entities.append([BookEntity.from_list_entity(ent).get_shortname() for ent in k])
    interactions.append(v)
    
interactions_df = pd.DataFrame({'entities':entities, 'interactions' : interactions})

In [134]:
interactions_df.head()

,entities,interactions
0,"[M Rênal, Abbé Julien Sorel]","[[ne, vous, attendez, point, à, trouver, en, f..."
1,"[M Rênal, Vicaire Maslon]","[[je, ne, trouve, quant, à, moi, qu, une, chos..."
2,"[M Rênal, M Valenod, Abbé Chélan]","[[tels, sont, les, faits, qui, commentés, exag..."
3,"[M Rênal, M Valenod]","[[les, reproches, de, m, de, rênal, et, surtou..."
4,"[M Rênal, Dieu]","[[, il, pourrait, bien, nous, enlever, celui, ..."
